# 2. Data Collection and Filtering
#### Angela Jiang, Alexander Lin, Jason Shen

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from datetime import timedelta
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.linear_model import LogisticRegressionCV as LogRegCV
import math
import string 
from six.moves.html_parser import HTMLParser
import urllib2
import json
import time
from functools import wraps
from copy import deepcopy
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Angela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Collection

In [495]:
def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck, e:
                    msg = "%s, Retrying in %d seconds..." % (str(e), mdelay)
                    if logger:
                        logger.warning(msg)
                    else:
                        print msg
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry

@retry(urllib2.HTTPError, tries=4, delay=3, backoff=2)
def get_url_with_retry(url):
    """ Makes API call to url and returns json response """
    response = urllib2.urlopen(url)
    text = response.read()
    test = json.loads(text)
    response.close()
    return test

def make_url(year, month, api_key):
    """ Returns string of API url to retrieve New York Times articles from the given year and month """
    return 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(year, month, api_key)

def get_news_by_time(df, year, month, api_key):
    """ Makes API call to retrive New York Times articles from the given year and month
    df: pandas dataframe with columns 'headline', 'paragraph', 'pub_date'
    Returns updated dataframe df with the 'headline', 'paragraph', 'pub_date' info
    of the retrieved articles """
    df_index = df.shape[0]
    
    # set URL with parameters of year and month [earliest first]
    url = make_url(year, month, api_key)
    # initial API call
    returned = get_url_with_retry(url)

    # get num results
    num_results = len(returned['response']['docs'])
    print 'Number of total results for {}/{}: {}'.format(month, year, num_results)

    articles_month_count = 0

    # for each of the results of the month, add results to dataframe
    for i, result in enumerate(returned['response']['docs']):
        # only look at articles, not multimedia or blog posts
        if type(result) is tuple:
            print i, result
            result = result[1]
        if result['document_type'] == 'article':
            if 'main' in result['headline']:
                headline = result['headline']['main']
            else:
                headline = result['lead_paragraph']
            df.loc[df_index] = [headline, result['lead_paragraph'], result['pub_date']]
            df_index += 1
            articles_month_count += 1

    print 'Number of article results for {}/{}: {}'.format(month, year, articles_month_count)

    return df

In [429]:
# creates dataframe to store data from 10 years
ten_year_data = pd.DataFrame(columns=('headline', 'paragraph', 'pub_date'))

# stores constants
api_key = 'd5a1e6fab7f04a10b2d3844b1b32b1ba'
start_year = 2006
start_month = 11
end_year = 2016
end_month = 11

In [439]:
# calculates number of calls (= number of months) that need to be made to the API function
num_calls = (end_year - start_year)*12 + end_month  - start_month + 1
year = start_year
month = start_month

# make API call for each year/month
for call in range(num_calls):
    ten_year_data = get_news_by_time(ten_year_data, year, month, api_key)
    
    # to next month
    month += 1
    
    # reset month, update year if after december
    if month == 13:
        month = 1
        year += 1

Number of total results for 2/2014: 7742
Number of article results for 2/2014: 4767
Number of total results for 3/2014: 8088
Number of article results for 3/2014: 5345
Number of total results for 4/2014: 7765
Number of article results for 4/2014: 5168
Number of total results for 5/2014: 8026
Number of article results for 5/2014: 5525
Number of total results for 6/2014: 8157
Number of article results for 6/2014: 5389
Number of total results for 7/2014: 7405
Number of article results for 7/2014: 5022
Number of total results for 8/2014: 7080
Number of article results for 8/2014: 4858
Number of total results for 9/2014: 8212
Number of article results for 9/2014: 5162
Number of total results for 10/2014: 8403
Number of article results for 10/2014: 5437
Number of total results for 11/2014: 7566
Number of article results for 11/2014: 5223
Number of total results for 12/2014: 7375
Number of article results for 12/2014: 4934
Number of total results for 1/2015: 7207
Number of article results for

KeyboardInterrupt: 

In [493]:
data_to_9_2015 = ten_year_data.reset_index(drop=True)
data_to_9_2015.head(n=5)
data_to_9_2015.to_excel('data_to_9_2015_2.xlsx')

#### Note:
* We did this data collection in batches because sometimes the API calls would time out, and we exported the dataframe from each batch to Excel (smaller file size than csvs). We then consolidated the batches of Excel documents into a single Excel document and saved it as '10_year_data.xlsx'

## Data Filtering

In [22]:
# read in full article data over the past 10 years
df = pd.read_excel('10_year_data.xlsx')
df['pub_date'] = pd.DatetimeIndex(df['pub_date']).normalize()
print 'Number of total articles:', df.shape[0]

Number of total articles: 655600


In [124]:
def filter_data(data, option = 'headline', date_filter = None, word_filter = None, pos = None, stem = False):
    """ Helper function that filters data by dates and/or words
    data = pandas dataframe with columns 'pub_date', 'headline', 'paragraph'
    option = 'headline' or 'paragraph' (default = 'headline')
    date_filer = (start date, end date) inclusive; else if default = None, then no filter 
    word_filter = list of words to filter OPTION by (CASE SENSITIVE); default is None
    pos = list of parts of speech tags (default = None)
    stem = boolean, whether or not to stem (default = False)"""
    
    h = HTMLParser()
    
    # filter by dates
    if date_filter is not None:
        start_date, end_date = date_filter
        filtered_data = data[(data['publish_date'] >= start_date) & (data['publish_date'] <= end_date)]
    else:
        filtered_data = data
        
    # filter by words
    if word_filter is not None:
        idx_to_drop = [] # store indices of rows that do not contain filter words
        
        # for every article
        for i in range(filtered_data.shape[0]):
            text = filtered_data.iloc[i][option]
            
            # iterates through each filter word
            filter_flag = 0
            # if there is no text (type is not string)
            if isinstance(text, basestring) is False:
                idx_to_drop.append(i)
                continue
            for word in word_filter:
                if word in text:
                    filter_flag = 1
                    break
            # if no filter words in text, drop
            if filter_flag == 0:
                idx_to_drop.append(i)
    
    # drops rows without words in filter
    filtered_data = filtered_data.drop(filtered_data.index[idx_to_drop])
    
    # remove caps
    filtered_data[option] = filtered_data[option].str.lower()
    
    # remove punctuation
    # remove html encoding puncuation from old news
    html_encoding = ['&#8217;', '&#8212;', '&#038;', '&#8230;', '&#8220;', '&#8221;']
    for i in range(filtered_data.shape[0]):
        for encoding in html_encoding:
            if encoding in filtered_data.iloc[i][option]:
                index = filtered_data.index[i]
                filtered_data.loc[index, option] = filtered_data.loc[index, option].replace(encoding, h.unescape(encoding))
    
    # remove other punctuation
    punctuation = list(',.!@#$%^&*()\'\"`:;?' + u'\u2018' + u'\u2019')
    for c in punctuation:
        filtered_data[option] = filtered_data[option].str.replace(c, '')
        
    return filtered_data

In [1]:
# filter by Apple-related key words in the headlines
filtered_df = filter_data(df, word_filter = ['Apple', 'AAPL', 'iPhone', 'iPod', 'MacBook'], option = 'headline')
print 'Number of articles after filtering', filtered_df.shape[0]

Number of articles after filtering: 1430


In [5]:
# export to Excel
filtered_df.to_excel('10_year_filtered_data.xlsx')